In [281]:
colors = [['R','G','G','R','R'],
          ['R','R','G','R','R'],
          ['R','R','G','G','R'],
          ['R','R','R','R','R']]
measurements = ['G','G','G','G','G']
motions = [[0,0],[0,1],[1,0],[1,0],[0,1]]


In [45]:
def show(p):
    rows = ['[' + ','.join(map(lambda x: '{0:.5f}'.format(x),r)) + ']' for r in p]
    print '[' + ',\n '.join(rows) + ']'

In [276]:
def move(P, motion, p_move):
    #get dimensions of p
    height_p = len(P)
    columns_p = len(P[0])
    
    p_dont_move = 1.0 - p_move
    
    
    #init new result based on the probability that the robot doesnt move and the prior probability
    q = [[P[row][col] * p_dont_move for col in range(columns_p)] for row in range(height_p)]

    total_prob = 0
    
    #loop through each row and column and find the appropriate value in the original matrix
    for i in range(height_p):
        for ii in range(columns_p):
            #shift the index to the left for forward motion and do the right for 
            #backwards motion. Truncate index at len and and wrap
            #similarly shift it down and up for vertical motion
            p_index_y = (i - motion[0]) % height_p
            p_index_x = (ii - motion[1]) % columns_p
            
            #the original prior probability
            p_value = P[p_index_y][p_index_x]
            
            q[i][ii] += p_value * p_move
            total_prob += q[i][ii]
    
    #normalize
    q = [[q[row][col]/total_prob for col in range(columns_p)] for row in range(height_p)]
    
    return q

In [272]:
def sense(P, sensor_value, p_sensor_right, world):
     #get dimensions of p
    height_p = len(P)
    columns_p = len(P[0])
    
    #init new result
    p_sensor_wrong = 1.0 - p_sensor_right
    q = [[0 for col in range(columns_p)] for row in range(height_p)]
    
    #loop over each column and add the probability based on a correct sensor setting
    total_prob = 0
    for i in range(height_p):
        for j in range(columns_p):
            
            if world[i][j] == sensor_value: # call this a hit
                q[i][j] += P[i][j] * p_sensor_right
            else:
                q[i][j] += P[i][j] * p_sensor_wrong
            total_prob += q[i][j]
            
    #normalize
    q = [[q[row][col]/total_prob for col in range(columns_p)] for row in range(height_p)] 

    return q

In [279]:
def localize(colors,measurements,motions,sensor_right,p_move):
    # initializes p to a uniform distribution over a grid of the same dimensions as colors
    pinit = 1.0 / float(len(colors)) / float(len(colors[0]))
    P = [[pinit for row in range(len(colors[0]))] for col in range(len(colors))]
    
    for m, s in zip(motions, measurements): 
        P = move(P, m, p_move)
        P = sense(P, s, sensor_right, colors)
    return P

In [280]:
p = localize(colors,measurements,motions,sensor_right = .8, p_move = 1)
show(p) # displays your answer

[[0.03333,0.03333,0.03333],
 [0.13333,0.13333,0.53333],
 [0.03333,0.03333,0.03333]]


In [282]:
p = localize(colors,measurements,motions,sensor_right = 0.7, p_move = 0.8)
show(p) # displays your answer

[[0.01106,0.02464,0.06800,0.04472,0.02465],
 [0.00715,0.01017,0.08697,0.07988,0.00935],
 [0.00740,0.00894,0.11273,0.35351,0.04066],
 [0.00911,0.00715,0.01435,0.04313,0.03643]]


In [ ]:
# [[0.01105, 0.02464, 0.06799, 0.04472, 0.02465],
#  [0.00715, 0.01017, 0.08696, 0.07988, 0.00935],
#  [0.00739, 0.00894, 0.11272, 0.35350, 0.04065],
#  [0.00910, 0.00715, 0.01434, 0.04313, 0.03642]]